In [1339]:
"""
Скрипт для создания отчета по форме БРИТ
"""

'\nСкрипт для создания отчета по форме БРИТ\n'

In [1340]:
import pandas as pd

In [1341]:
def count_value(group:pd.Series,target_value:str):
    """
    Функция для группировки по конкретному значению
    Возвращает количество значений target_value в группе
    """
    # считаем сколько значений подходят по условие
    count_group = group.str.contains(target_value)
    return sum(count_group)


In [1342]:
check_cols = {'Статус_учёба','Статус_Сиротство','Статус_Уровень_здоровья'} # проверяемые колонки
dct_name_sheet = dict() # словарь где ключ это названия листа а значение датафрейм на основе которого был произведен подсчет

In [1343]:
df = pd.read_excel('data/Общий файл.xlsx',dtype=str)
df.fillna('Нет статуса',inplace=True) # заполняем Наны
group_main_df = pd.DataFrame(index=list(df['Файл'].unique()))
group_main_df.index.name = 'Файл'

C:\Users\1\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [1344]:
# Отбрасываем на всякий случай отчисленных
df = df[df['Статус_учёба']!= 'Отчислен']

In [1345]:
# Создаем датафрейм с сиротами
orphans_df = df[df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]

dct_name_sheet['Сироты'] = orphans_df # добавляем в словарь
orphans_group_df = orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'}) # создаем базовый 
group_main_df = group_main_df.join(orphans_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Дети-сироты'},inplace=True)

In [1346]:
group_main_df

,Дети-сироты
Файл,
Железнодорожного транспорта,9
Энергоснабжения,13
Металлообработки,29
Хоринский филиал,4


In [1347]:
# Создаем датафрейм с сиротами
invalid_df = df[df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]

dct_name_sheet['Инвалиды'] = invalid_df # добавляем в словарь
invalid_group_df = invalid_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'}) # создаем базовый 
group_main_df = group_main_df.join(invalid_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Инвалиды'},inplace=True)

In [1348]:
group_main_df

,Дети-сироты,Инвалиды
Файл,,
Железнодорожного транспорта,9,10
Энергоснабжения,13,10
Металлообработки,29,22
Хоринский филиал,4,3


In [1349]:
# Создаем датафрейм с получателями социальной стипендии
print(df.shape)
soc_benefit_df = df[df['Статус_соц_стипендия'].isin(['да'])]
print(soc_benefit_df.shape)
dct_name_sheet['Соц. стипендия Все'] = soc_benefit_df # добавляем в словарь





(252, 53)
(30, 53)


In [1350]:
# получаем малоимущих
poor_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Соц_положение_семьи'].isin(['Малоимущая'])]
dct_name_sheet['Соц. стипендия Малоим.'] = poor_soc_benefit_df
poor_soc_benefit_group_df = poor_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Соц_положение_семьи':'count'})
group_main_df = group_main_df.join(poor_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Соц_положение_семьи':'Соц. стипендия малоимущие'},inplace=True)


In [1351]:
# получаем детей сирот
orphans_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Соц. стипендия Сироты'] = orphans_soc_benefit_df
orphans_soc_benefit_group_df = orphans_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_main_df = group_main_df.join(orphans_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Соц. стипендия дети-сироты'},inplace=True)

In [1352]:
# получаем инвалидов
invalid_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]
dct_name_sheet['Соц. стипендия Инвалиды'] = invalid_soc_benefit_df
invalid_soc_benefit_group_df = invalid_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'})
group_main_df = group_main_df.join(invalid_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Соц. стипендия инвалиды'},inplace=True)

In [1353]:
# Создаем датафрейм с получателями бесплатного питания
print(df.shape)
eating_df = df[df['Статус_питание'].isin(['получает компенсацию за питание','питается в БРИТ'])]
print(eating_df.shape)
dct_name_sheet['Соц. стипендия Все'] = eating_df # добавляем в словарь

(252, 53)
(60, 53)


In [1354]:
# получаем малоимущих
poor_eating_df = eating_df[eating_df['Статус_Соц_положение_семьи'].isin(['Малоимущая'])]
dct_name_sheet['Питание Малоим.'] = poor_eating_df
poor_eating_group_df = poor_eating_df.groupby(by=['Файл']).agg({'Статус_Соц_положение_семьи':'count'})
group_main_df = group_main_df.join(poor_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Соц_положение_семьи':'Питание малоимущие'},inplace=True)

In [1355]:
# получаем сирот
orphans_eating_df = eating_df[eating_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Питание Сироты'] = orphans_eating_df
orphans_eating_group_df = orphans_eating_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_main_df = group_main_df.join(orphans_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Питание сироты'},inplace=True)

In [1356]:
# получаем инвалидов
invalid_eating_df = eating_df[eating_df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]
dct_name_sheet['Питание Инвалиды'] = invalid_eating_df
invalid_eating_group_df = invalid_eating_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'})
group_main_df = group_main_df.join(invalid_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Питание инвалиды'},inplace=True)

In [1357]:
# получаем СВО
svo_eating_df = eating_df[eating_df['Статус_родитель_СВО'].isin(['да'])]
dct_name_sheet['Питание СВО'] = svo_eating_df
svo_eating_group_df = svo_eating_df.groupby(by=['Файл']).agg({'Статус_родитель_СВО':'count'})
group_main_df = group_main_df.join(svo_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_родитель_СВО':'Питание СВО'},inplace=True)

In [1358]:
# Создаем датафрейм с проживающими в общежитии
print(df.shape)
dormitory_df = df[df['Статус_общежитие'].isin(['да'])]
print(dormitory_df.shape)
dct_name_sheet['Общежитие Все'] = dormitory_df # добавляем в словарь
dormitory_group_df = dormitory_df.groupby(by=['Файл']).agg({'Статус_общежитие':'count'})
group_main_df = group_main_df.join(dormitory_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_общежитие':'Общежитие Всего'},inplace=True)

(252, 53)
(71, 53)


In [1359]:
# получаем не сирот и сохраняем для списка
not_orphans_dormitory_df = dormitory_df[~dormitory_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Общежитие Кроме сирот'] = not_orphans_dormitory_df


In [1360]:
# получаем сирот
# получаем сирот
orphans_dormitory_df = dormitory_df[dormitory_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Питание Сироты'] = orphans_dormitory_df
orphans_dormitory_group_df = orphans_dormitory_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_main_df = group_main_df.join(orphans_dormitory_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Общежитие сироты'},inplace=True)

In [1361]:
group_main_df

,Дети-сироты,Инвалиды,Соц. стипендия малоимущие,Соц. стипендия дети-сироты,Соц. стипендия инвалиды,Питание малоимущие,Питание сироты,Питание инвалиды,Питание СВО,Общежитие Всего,Общежитие сироты
Файл,,,,,,,,,,,
Железнодорожного транспорта,9,10,1.0,3,1.0,1.0,3,3,2,12,5
Энергоснабжения,13,10,NaN,4,NaN,NaN,4,3,3,17,7
Металлообработки,29,22,NaN,8,NaN,NaN,8,7,7,37,15
Хоринский филиал,4,3,NaN,1,1.0,2.0,1,1,2,5,2


In [1362]:
group_main_df.fillna(0,inplace=True) # заполняем наны
group_main_df=group_main_df.astype(int) # приводим к инту
sum_row = group_main_df.sum(axis=0) # суммируем колонки
group_main_df.loc['Итого'] = sum_row # добавляем суммирующую колонку

In [1363]:
group_main_df

,Дети-сироты,Инвалиды,Соц. стипендия малоимущие,Соц. стипендия дети-сироты,Соц. стипендия инвалиды,Питание малоимущие,Питание сироты,Питание инвалиды,Питание СВО,Общежитие Всего,Общежитие сироты
Файл,,,,,,,,,,,
Железнодорожного транспорта,9,10,1,3,1,1,3,3,2,12,5
Энергоснабжения,13,10,0,4,0,0,4,3,3,17,7
Металлообработки,29,22,0,8,0,0,8,7,7,37,15
Хоринский филиал,4,3,0,1,1,2,1,1,2,5,2
Итого,55,45,1,16,2,3,16,14,14,71,29


In [1364]:
group_orphans_main_df = pd.DataFrame(index=list(df['Файл'].unique())) # Базовый датафрейм для сирот


In [1365]:
# Считаем общее количество
all_orphans_group_df = orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'}) # создаем базовый 
group_orphans_main_df = group_orphans_main_df.join(all_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Всего'},inplace=True)

In [1366]:
# считаем академ
akadem_orphans_df = orphans_df[orphans_df['Статус_учёба'].str.contains('Академический отпуск')]
dct_name_sheet['Сироты Академ'] = akadem_orphans_df
akadem_orphans_group_df = akadem_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(akadem_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них в академическом отпуске'},inplace=True)

In [1367]:
# считаем постинтернатное сопровождение
postinternat_orphans_df = orphans_df[orphans_df['Статус_Сиротство'].str.contains('постинтернатное сопровождение')]
dct_name_sheet['Сироты постинтернат'] = postinternat_orphans_df
postinternat_orphans_group_df = postinternat_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(postinternat_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них постинтернат'},inplace=True)

In [1368]:
# считаем сирот на гособеспечении
print(orphans_df.shape)
gos_orphans_df = orphans_df[orphans_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении'
                                                     ])]
print(gos_orphans_df.shape)
dct_name_sheet['Сироты гособеспечение'] = gos_orphans_df
gos_orphans_group_df = gos_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(gos_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них на гособеспечении'},inplace=True)

(55, 53)
(41, 53)


In [ ]:
# считаем сирот на гособеспечении
print(orphans_df.shape)
gos_orphans_df = orphans_df[orphans_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении'
                                                     ])]
print(gos_orphans_df.shape)
dct_name_sheet['Сироты гособеспечение'] = gos_orphans_df
gos_orphans_group_df = gos_orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_orphans_main_df = group_orphans_main_df.join(gos_orphans_group_df) # добавляем в свод
group_orphans_main_df.rename(columns={'Статус_Сиротство':'Из них на гособеспечении'},inplace=True)

In [1369]:
group_orphans_main_df

,Всего,Из них в академическом отпуске,Из них постинтернат,Из них на гособеспечении
Железнодорожного транспорта,9,2,4,7
Энергоснабжения,13,3,6,10
Металлообработки,29,7,14,22
Хоринский филиал,4,2,2,2
